download

In [ ]:
! pip install geemap

In [1]:
! pip install -U -q earthengine-api

     |████████████████████████████████| 229 kB 28.8 MB/s 


In [ ]:
! pip install -U -q folium

import

In [335]:
import ee
import folium
import geemap
import numpy as np
import pandas as pd
from sklearn.cluster import KMeans
from sklearn.preprocessing import minmax_scale
from sklearn.decomposition import PCA
import matplotlib.pyplot as plt

location selection

In [ ]:
ee.Authenticate()

In [5]:
ee.Initialize()

In [308]:
#adding rastors from earth engine to folium map
def add_ee_layer(self, ee_image_object, vis_params, name):
  map_id_dict = ee.Image(ee_image_object).getMapId(vis_params)
  folium.raster_layers.TileLayer(
      tiles=map_id_dict['tile_fetcher'].url_format,
      attr='Map Data &copy; <a href="https://earthengine.google.com/">Google Earth Engine</a>',
      name=name,
      overlay=True,
      control=True
  ).add_to(self)

folium.Map.add_ee_layer = add_ee_layer


#soecfy terrain area you want to segment (limited number of pixels)
bound_ = [[6.219817916110255,36.75729987187895]
,[6.421348372653224,36.75729987187895]
,[6.421348372653224,36.849939905468865]
,[6.219817916110255,36.849939905468865]
,[6.219817916110255,36.75729987187895]]



point_ = bound_[0]
point = ee.Geometry.Point(point_)
bound = ee.Geometry.Polygon(bound_)

importing satelite imagery

In [295]:
#importing the whole collection of satelite images
satlite_dataset = ee.ImageCollection('LANDSAT/LC08/C01/T1_SR')
#selecting the image with the best covrage for our location of interest
sat_image = satlite_dataset.filterBounds(point).sort('CLOUD_COVER').first()

In [309]:

# adding sat image to map
viz_params = {
    "min":0,
    "max":3000,
    "bands":["B4","B3","B2"]
}

map1 = folium.Map(location=[point_[1],point_[0]],zoom_start=12)
map1.add_ee_layer(sat_image.clip(bound),viz_params,name="sat_image")

In [310]:
map1

k-means clustering

In [351]:
#converting the rastor into a numpy array
rastor = geemap.ee_to_numpy(sat_image,region=bound,default_value=0)

In [350]:
w,h,f=rastor.shape
rastor = rastor.reshape((w*h,f))

ValueError: ignored

In [345]:
#convert to pandas data-frame
#each pixel is row and each band is a column
data = pd.DataFrame(rastor,columns=sat_image.bandNames().getInfo())
data = data.drop(["sr_aerosol","pixel_qa","radsat_qa"],axis=1) #drop unused bands

In [346]:
#normlize data
data = minmax_scale(data)

In [347]:
#pca data reduction
n_components=5
pca = PCA(n_components)
data_r = pd.DataFrame(pca.fit_transform(data))

In [359]:
data_train = data_r.sample(frac=0.5)

In [365]:
#applying k-means
n_clusters = 4
km = KMeans(n_clusters,random_state=0)
km.fit(data_r)
classes = km.predict(data_r)

In [366]:
classes

array([2, 2, 2, ..., 2, 2, 2], dtype=int32)

In [367]:
classes = classes.reshape((w,h))

In [368]:
#classes coloring function
hue = {
    0:[82/255, 182/255, 154/255],
    1:[30/255, 96/255, 145/255],
    2:[217/255, 237/255, 146/255],
    3:[177/255, 167/255, 166/255]
}
f= lambda i:hue[i]

In [369]:
#adding k-means segmentation to the map 
map1 = folium.Map(location=[point_[1],point_[0]],zoom_start=12)
map1.add_ee_layer(sat_image.clip(bound),viz_params,name="sat_image")
folium.raster_layers.ImageOverlay(classes,bounds=[[bound_[2][1],bound_[2][0]],[bound_[0][1],bound_[0][0]]],colormap=f,name="sklearn cluster").add_to(map1)
folium.LayerControl().add_to(map1)
display(map1)
